<a href="https://colab.research.google.com/github/ManelSoengas/NLP/blob/main/NLP_new_light.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




```
https://www.geeksforgeeks.org/sentiment-classification-using-bert/




# <font color="#7C220F"> **Install and load libraries. Load the data set**. </font>

---



In [ ]:
!pip install datasets


import pandas as pd
import re
import tensorflow as tf

from transformers import BertTokenizer, TFBertForSequenceClassification
from datasets import load_dataset
from sklearn.metrics import classification_report

dataset = load_dataset("carblacac/twitter-sentiment-analysis")
df_train = pd.DataFrame(dataset['train'])
df_test = pd.DataFrame(dataset['test'])
df_validation = pd.DataFrame(dataset['validation'])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for carblacac/twitter-sentiment-analysis contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/carblacac/twitter-sentiment-analysis
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the ne

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/149985 [00:00<?, ? examples/s]

...

Creating json from Arrow format:   0%|          | 0/30 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/62 [00:00<?, ?ba/s]

Generating train split:   0%|          | 0/119988 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/29997 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/61998 [00:00<?, ? examples/s]

# <font color="#7C220F"> **Data set visualization**. </font>

---
**Most representative data**


In [ ]:
# Sample of the first five rows
print(df_train.head(5))


                                                text  feeling
0  @fa6ami86 so happy that salman won.  btw the 1...        0
1  @phantompoptart .......oops.... I guess I'm ki...        0
2  @bradleyjp decidedly undecided. Depends on the...        1
3  @Mountgrace lol i know! its so frustrating isn...        1
4  @kathystover Didn't go much of any where - Lif...        1


In [ ]:
# Shows the columns and their data types
print(df_train.dtypes)


text       object
feeling     int64
dtype: object


In [ ]:
# Statistical summary
print(df_train.describe())


             feeling
count  119988.000000
mean        0.500208
std         0.500002
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         1.000000


In [ ]:
# DataFrame Overview
print(df_train.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119988 entries, 0 to 119987
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   text     119988 non-null  object
 1   feeling  119988 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.8+ MB
None


In [ ]:
# Count unique values of target class
print(df_test['feeling'].value_counts())


1    31029
0    30969
Name: feeling, dtype: int64


In [ ]:
print(df_train.isna().sum())



text       0
feeling    0
dtype: int64


# <font color="#7C220F"> **Preprocessing**. </font>




---

In [ ]:


def clear_text(text):
    """
    Remove user mentions and URLs from a string.

     Parameters:
     text (str): Text to clear.

     Returns:
     str: Cleaned text.
    """
    # Delete user mentions (anything that starts with @ followed by alphanumeric characters)
    text = re.sub(r'@\w+', '', text)

    # Remove URLs (strings starting with http, https or www)
    text = re.sub(r'http\S+|https\S+|www\S+', '', text)

    return text




In [ ]:
# clear_text function call

df_train['text'] = df_train['text'].apply(clear_text)
df_test['text'] = df_test['text'].apply(clear_text)
df_validation['text'] = df_validation['text'].apply(clear_text)

In [ ]:


def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove unnecessary repetitions of characters like .,!?
     # This line replaces any group of these characters with the same character only once
    text = re.sub(r'([.,!?])\1+', r'\1', text)
    # Remove or replace other special characters
     # Here characters other than letters, numbers, spaces or the common characters .,!?;
    text = re.sub(r"[^a-zA-Z0-9.,!?;]+", ' ', text)
    return text

# Apply the preprocessing function to each DataFrame

df_train['text'] = df_train['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)
df_validation['text'] = df_validation['text'].apply(preprocess_text)







In [ ]:

def clean_text(text):

    # Remove non-ASCII characters
    text = text.encode('ascii', 'ignore').decode('ascii')

    # Remove specific punctuation and special characters
    chars_to_remove = ['Ã', '±', 'ã', '¼', 'â', '»', '§', '&', '$']
    for char in chars_to_remove:
        text = text.replace(char, '')

   # Remove punctuation (optional, uncomment the next line if you also want to remove all punctuation)
     # text = re.sub(r'[^\w\s]', '', text)

     # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply the function to the 'text' column of each dataframe

df_train['text'] = df_train['text'].apply(clean_text)
df_test['text'] = df_test['text'].apply(clean_text)
df_validation['text'] = df_validation['text'].apply(clean_text)


In [ ]:
print(df_test.head())


                                                text  feeling
0                 .yeahhh. i m 39 tweets from 1,600!        1
1         aww. poor baby! on your only real day off.        0
2  with my refunded 225 australian ticket price i...        0
3  it s fine. today sucks just because me those t...        0
4  im just chilling on psp and stuff, but sitting...        0


In [ ]:
#Save preprocessed dataframes

df_train.to_csv('/content/df_train.csv', index=False)
df_test.to_csv('/content/df_test.csv', index=False)
df_validation.to_csv('/content/df_validation.csv', index=False)


In [ ]:
def max_sentence_length(df):
    df['sentence_length'] = df['text'].apply(lambda x: len(x.split()))
    max_length = df['sentence_length'].max()
    return max_length, df['sentence_length'].describe()

# Calculate and display the maximum length and statistical description for each DataFrame

max_length_train, desc_train = max_sentence_length(df_train)
max_length_test, desc_test = max_sentence_length(df_test)
max_length_validation, desc_validation = max_sentence_length(df_validation)

print("Maximum length in df_train:", max_length_train)
print(desc_train)
print("\nMaximum length inn df_test:", max_length_test)
print(desc_test)
print("\nMaximum length in df_validation:", max_length_validation)
print(desc_validation)



Maximum length in df_train: 58
count    119988.000000
mean         13.165942
std           7.197486
min           0.000000
25%           7.000000
50%          12.000000
75%          19.000000
max          58.000000
Name: sentence_length, dtype: float64

Maximum length inn df_test: 45
count    61998.000000
mean        13.169054
std          7.183799
min          0.000000
25%          7.000000
50%         12.000000
75%         19.000000
max         45.000000
Name: sentence_length, dtype: float64

Maximum length in df_validation: 38
count    29997.000000
mean        13.184585
std          7.197051
min          0.000000
25%          7.000000
50%         12.000000
75%         19.000000
max         38.000000
Name: sentence_length, dtype: float64


# <font color="#7C220F"> **Loading of the tokenized. Dataframe encoding**. </font>




---


In [ ]:


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
max_len= 60  # Sentence Maximum lenght.

# Tokenize and get features from sentences
X_train_features = tokenizer.batch_encode_plus(df_train['text'].tolist(),
                                              padding=True,
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')

X_val_features = tokenizer.batch_encode_plus(df_validation['text'].tolist(),
                                              padding=True,
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')

X_test_features = tokenizer.batch_encode_plus(df_test['text'].tolist(),
                                              padding=True,
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Tokenizar el conjunto de datos
X_train_features = tokenizer.batch_encode_plus(df_train['text'].tolist(),
                                               padding=True,
                                               truncation=True,
                                               max_length=max_len,
                                               return_tensors='tf')

# Ver los tokens y sus respectivos tensores
for i, (tokens, tensor) in enumerate(zip(X_train_features['input_ids'], X_train_features['input_ids'].numpy())):
    print("Texto original:", df_train['text'].iloc[i])
    print("Tokens:", tokenizer.convert_ids_to_tokens(tokens))
    print("Tensor:", tensor)
    if i == 1:  # Limitar la visualización a solo un par de ejemplos
        break


# <font color="#7C220F"> **Loading the BERT pre-trained model**. </font>


---




In [ ]:


model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# <font color="#7C220F"> **Model configuration**. </font>


---


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
# Obtaining the data of the target variable

y_train = df_train['feeling']

y_test = df_test['feeling']

y_val = df_validation['feeling']

# <font color="#7C220F"> **Training Model**. </font>


---


In [ ]:
# token_type_ids will not be necessary
history = model.fit(
    [X_train_features['input_ids'], X_train_features['token_type_ids'], X_train_features['attention_mask']],
    y_train,
    validation_data=(
      [X_val_features['input_ids'], X_val_features['token_type_ids'], X_val_features['attention_mask']],y_val),
    batch_size=32,
    epochs=3
)

Epoch 1/3
3750/3750 [==============================] - 1643s 424ms/step - loss: 0.3896 - accuracy: 0.8209 - val_loss: 0.3445 - val_accuracy: 0.8507
Epoch 2/3
3750/3750 [==============================] - 1578s 421ms/step - loss: 0.2887 - accuracy: 0.8776 - val_loss: 0.3424 - val_accuracy: 0.8523
Epoch 3/3
3750/3750 [==============================] - 1578s 421ms/step - loss: 0.1934 - accuracy: 0.9227 - val_loss: 0.4026 - val_accuracy: 0.8474


In [ ]:
#Evaluate the model on the test data

test_loss, test_accuracy = model.evaluate(
    [X_test_features['input_ids'], X_test_features['token_type_ids'], X_test_features['attention_mask']],
    y_test
)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

1938/1938 [==============================] - 256s 132ms/step - loss: 0.4274 - accuracy: 0.8408
Test loss: 0.4274110198020935, Test accuracy: 0.8408174514770508


# <font color="#7C220F"> **Prediction on the test set**. </font>


---

In [ ]:

pred = model.predict(
    [X_test_features['input_ids'], X_test_features['token_type_ids'], X_test_features['attention_mask']])

# pred is of type TFSequenceClassifierOutput
logits = pred.logits

# Use argmax along the appropriate axis to get the predicted labels
pred_labels = tf.argmax(logits, axis=1)

# Convert the predicted labels to a NumPy array
pred_labels = pred_labels.numpy()

label = {
    1: 'Positive',
    0: 'Negative'
}

# Map the predicted labels to their corresponding strings using the label dictionary
pred_labels = [label[i] for i in pred_labels]
Actual = [label[i] for i in y_test]

print('Predicted Label :', pred_labels[:10])
print('Actual Label    :', Actual[:10])

1938/1938 [==============================] - 253s 129ms/step
Predicted Label : ['Positive', 'Negative', 'Positive', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Positive', 'Negative']
Actual Label    : ['Positive', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Positive', 'Negative']


In [ ]:

print("Classification Report: \n", classification_report(Actual, pred_labels))

Classification Report: 
               precision    recall  f1-score   support

    Negative       0.82      0.87      0.84     30969
    Positive       0.86      0.81      0.84     31029

    accuracy                           0.84     61998
   macro avg       0.84      0.84      0.84     61998
weighted avg       0.84      0.84      0.84     61998



# <font color="#7C220F"> **Save the model and tokenizer**. </font>


---

In [ ]:
path = '/content/'
# Save tokenizer
tokenizer.save_pretrained(path +'/tokenizer')

# Save model
model.save_pretrained(path +'/model')

# <font color="#7C220F"> **Test the model**. </font>


---

In [ ]:
def Get_sentiment(tweet, Tokenizer=tokenizer, Model=model):
    # Convert tweet to a list if it's not already a list
    if not isinstance(tweet, list):
        tweet = [tweet]

    Input_ids, Token_type_ids, Attention_mask = Tokenizer.batch_encode_plus(tweet,
                                                                             padding=True,
                                                                             truncation=True,
                                                                             max_length=60,
                                                                             return_tensors='tf').values()
    prediction = Model.predict([Input_ids, Token_type_ids, Attention_mask])

    # Use argmax along the appropriate axis to get the predicted labels
    pred_labels = tf.argmax(prediction.logits, axis=1)

    # Convert the TensorFlow tensor to a NumPy array and then to a list to get the predicted sentiment labels
    pred_labels = [label[i] for i in pred_labels.numpy().tolist()]
    return pred_labels

In [ ]:
tweet ='''I'm really good, :-).'''
Get_sentiment(tweet)

1/1 [==============================] - 0s 69ms/step


['Positive']